In [ ]:
import os
import shutil
import random

# Set your paths
original_dataset_dir = r"C:\Users\OneDrive\Documents\weapon6\dataset"  # e.g., 'dataset_raw'
base_dir = 'split_dataset'  # New organized dataset

# Create folders
for split in ['train', 'val', 'test']:
    for category in ['gun', 'knife', 'neg']:
        os.makedirs(os.path.join(base_dir, split, category), exist_ok=True)

# Split ratios
train_split = 0.7
val_split = 0.2
test_split = 0.1

# Categories
categories = ['gun', 'knife', 'neg']

for category in categories:
    category_path = os.path.join(original_dataset_dir, category)
    images = os.listdir(category_path)
    random.shuffle(images)

    total_images = len(images)
    train_count = int(total_images * train_split)
    val_count = int(total_images * val_split)

    # Split images
    train_images = images[:train_count]
    val_images = images[train_count:train_count + val_count]
    test_images = images[train_count + val_count:]

    # Move images
    for img in train_images:
        shutil.copy(os.path.join(category_path, img), os.path.join(base_dir, 'train', category, img))

    for img in val_images:
        shutil.copy(os.path.join(category_path, img), os.path.join(base_dir, 'val', category, img))

    for img in test_images:
        shutil.copy(os.path.join(category_path, img), os.path.join(base_dir, 'test', category, img))

print("Dataset organized successfully!")


Dataset organized successfully!


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

# Only rescale for validation and testing
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'split_dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    'split_dataset/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    'split_dataset/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 256 images belonging to 3 classes.
Found 73 images belonging to 3 classes.
Found 38 images belonging to 3 classes.


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Load the pre-trained model
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                                include_top=False,
                                                weights='imagenet')

base_model.trainable = False  # Freeze base model

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(3, activation='softmax')  # 3 classes: gun, knife, neg
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,339 (9.24 MB)

 Trainable params: 164,355 (642.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [4]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50
)

C:\Users\srava\AppData\Roaming\Python\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - accuracy: 0.6393 - loss: 0.8213 - val_accuracy: 0.8767 - val_loss: 0.2771
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.9475 - loss: 0.1783 - val_accuracy: 0.9726 - val_loss: 0.1014
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.9467 - loss: 0.1765 - val_accuracy: 0.8767 - val_loss: 0.2091
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.9553 - loss: 0.1713 - val_accuracy: 0.9452 - val_loss: 0.1123
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 22s 3s/step - accuracy: 0.9714 - loss: 0.0762 - val_accuracy: 0.9589 - val_loss: 0.0909
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 20s 3s/step - accuracy: 0.9861 - loss: 0.0496 - val_accuracy: 1.0000 - val_loss: 0.0337
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 32s 3s/step - accuracy: 0.9640 - loss: 0.0995 - val_accuracy: 0.9726 - val_loss: 0.0703
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 35s 4s/step - accuracy: 0.9914 - loss: 0.0298 - val_accuracy: 0.9726 - val_loss: 0.0662
Epoch 9/

In [5]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc*100:.2f}%")

# Save the model
model.save('weapon_detection_model.h5')


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 148ms/step - accuracy: 1.0000 - loss: 0.0081


Test accuracy: 100.00%


In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
# Load your saved model
model = tf.keras.models.load_model('weapon_detection_model.h5')
import os

test_dir = 'split_dataset/test'

class_labels = ['gun', 'knife', 'neg']

for category in os.listdir(test_dir):
    category_path = os.path.join(test_dir, category)
    for img_name in os.listdir(category_path):
        img_path = os.path.join(category_path, img_name)
        
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0

        prediction = model.predict(img_array)
        predicted_class = np.argmax(prediction)

        print(f"Actual: {category}, Predicted: {class_labels[predicted_class]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Actual: gun, Predicted: gun
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
Actual: gun, Predicted: gun
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
Actual: gun, Predicted: gun
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
Actual: gun, Predicted: gun
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
Actual: gun, Predicted: gun
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
Actual: gun, Predicted: gun
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
Actual: gun, Predicted: gun
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
Actual: gun, Predicted: gun
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
Actual: gun, Predicted: gun
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
Actual: gun, Predicted: gun
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
Actual: gun, Predicted: gun
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
Actual: gun, Predicted: gun
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
Actual: gun, Predicted: gun
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
Actual: gun, Predicted: gun
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
Actual: gun, Predicted: gu

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# Load the trained model
model = tf.keras.models.load_model('weapon_detection_model.h5')

# Define class labels
class_labels = ['gun', 'knife', 'neg']

# Open webcam
cap = cv2.VideoCapture(0)  # 0 = default camera

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame
    img = cv2.resize(frame, (224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    # Make prediction
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    confidence = np.max(prediction)

    # Display the prediction
    label = f"{class_labels[predicted_class]}: {confidence*100:.2f}%"
    color = (0, 255, 0) if class_labels[predicted_class] == 'neg' else (0, 0, 255)

    cv2.putText(frame, label, (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    cv2.imshow('Weapon Detection', frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11